In [155]:
import pandas as pd
import numpy as np
from Ames_Functions import OSMR, get_geoapify, get_dist, locator, find_min_dist
import pickle

## House Latlongs Lookup Section

In [157]:
df_RE_latest = pd.read_csv("data/Ames_Real_Estate_Data_Latest.csv")
df_EssentialLatLong = pd.read_csv("df_EssentialLatLong_v2.csv")

/var/folders/2n/5k95xl754cb_szsgjkmj33k80000gn/T/ipykernel_11049/1833454112.py:1: DtypeWarning: Columns (36,40,81,87) have mixed types. Specify dtype option on import or set low_memory=False.
  df_RE_latest = pd.read_csv("data/Ames_Real_Estate_Data_Latest.csv")


In [51]:
df_RE_latlongs.iloc[97,:].to_frame().T

,MapRefNo,GeoRefNo,Tier,Range,Prop_Addr,ZngCdPr,ZngCdSc,ZngOLPr,ZngOLSc,ClassPr_S,...,PA-PreD,PA-Strt,PA-StSfx,PA-PostD,PA-UnTyp,PA-UntNo,Date,Source,NmbrBRs,reverse_lookup_rd
1354,527359170.0,527359170,0,0,2989 MONROE DR,RL,NaN,NaN,NaN,RESIDENTIAL,...,NaN,MONROE,DR,NaN,NaN,NaN,31-Mar-22,Ames City Assessor,4.0,NaN


In [3]:
#Storing Latlongs using Geoapify

df_RE_latlongs = df_RE_latest.query('YrSold_YYYY == 2019').drop_duplicates()
latlongs = df_RE_latlongs.Prop_Addr.apply(lambda x: get_geoapify(x + ", Ames, Iowa"))

Latitude: 42.0748755, Longitude: -93.6352983
Latitude: 42.0748755, Longitude: -93.6352983
Latitude: 42.077415500442285, Longitude: -93.62992402210288
Latitude: 42.07740283436533, Longitude: -93.6300160515734
Latitude: 42.077345553486566, Longitude: -93.63103402570076
Latitude: 42.07734550206757, Longitude: -93.63110369843315
Latitude: 42.077344679363726, Longitude: -93.63221846215146
Latitude: 42.07734462794473, Longitude: -93.63228813488385
Latitude: 42.07734457652574, Longitude: -93.63235780761624
Latitude: 42.07734452510675, Longitude: -93.63242748034864
Latitude: 42.07734447368776, Longitude: -93.63249715308103
Latitude: 42.07734423366438, Longitude: -93.63337967431015
Latitude: 42.07734430130779, Longitude: -93.63344934702867
Latitude: 42.07734436895121, Longitude: -93.63351901974721
Latitude: 42.07734443659463, Longitude: -93.63358869246574
Latitude: 42.07734443659463, Longitude: -93.63358869246574
Latitude: 42.07734450423804, Longitude: -93.63365836518427
Latitude: 42.0768347838

In [ ]:
# #Storing Latlongs using Geoapify

# df_RE_latlongs = df_RE_latest.query('YrSold_YYYY == 2021').drop_duplicates()
# latlongs = df_RE_latlongs.Prop_Addr.apply(lambda x: get_geoapify(x + ", Ames, Iowa"))

In [8]:
df_RE_latest.query('YrSold_YYYY == 2019').duplicated().sum()

3

In [13]:
#Checking Latlongs with reverse geocoding
#latlongs.tail(2)
s_address = latlongs.apply(lambda x: locator.reverse(x).raw['address'])
s_address

92       {'road': 'Mcfarland Avenue', 'residential': 'Q...
93       {'road': 'Mcfarland Avenue', 'residential': 'Q...
96       {'house_number': '1407', 'road': 'Ledges Drive...
98       {'house_number': '1415', 'road': 'Ledges Drive...
100      {'house_number': '1503', 'road': 'Ledges Drive...
                               ...                        
22745    {'house_number': '500', 'road': 'Ken Maril Roa...
22747    {'house_number': '512', 'road': 'Ken Maril Roa...
22750    {'house_number': '608', 'road': 'Ken Maril Roa...
22755    {'house_number': '700', 'road': 'Ken Maril Roa...
22797    {'house_number': '406', 'road': 'East Lincoln ...
Name: Prop_Addr, Length: 901, dtype: object

In [78]:
#pd.json_normalize(s_address).join(pd.Series(s_address.index, name = 'RE_index')).query("index == 96 or index == 97")
s_road = pd.json_normalize(s_address).road.fillna("Monroe Drive")
s_road.to_frame()

,road
0,Mcfarland Avenue
1,Mcfarland Avenue
2,Ledges Drive
3,Ledges Drive
4,Ledges Drive
...,...
896,Ken Maril Road
897,Ken Maril Road
898,Ken Maril Road
899,Ken Maril Road


In [135]:
df_RE_latlongs = df_RE_latlongs.join(latlongs.str.split(",", expand = True).set_axis(["Lat", "Long"], axis = 1))

In [138]:
#df_RE_latlongs[["PA-Strt"]].str.lower()
df_RE_latlongs['reverse_lookup_rd'] = s_road.values
mask = df_RE_latlongs.apply(lambda x: x["PA-Strt"].lower() not in x.reverse_lookup_rd.lower(), axis = 1)
df_RE_latlongs.query('@mask')[["Prop_Addr", "reverse_lookup_rd", "Lat", "Long"]].to_csv("df_RE_latlongs2019_cleanup.csv")
#df_RE_latlongs["PA-Strt"].str.lower()

In [147]:
df_RE_latlongs.loc[df_latlong_corrections.index, ["Lat", "Long"]] = df_latlong_corrections[["Lat", "Long"]]
df_RE_latlongs.loc[df_latlong_corrections.index]

,MapRefNo,GeoRefNo,Tier,Range,Prop_Addr,ZngCdPr,ZngCdSc,ZngOLPr,ZngOLSc,ClassPr_S,...,PA-StSfx,PA-PostD,PA-UnTyp,PA-UntNo,Date,Source,NmbrBRs,reverse_lookup_rd,Lat,Long
11371,906329170.0,906329170,0,0,5116 SPRINGBROOK CIR,FS-RL,NaN,NaN,NaN,RESIDENTIAL,...,CIR,NaN,NaN,NaN,31-Mar-22,Ames City Assessor,2.0,6th Street,42.028568,-93.687674
11373,906329180.0,906329180,0,0,5122 SPRINGBROOK CIR,FS-RL,NaN,NaN,NaN,RESIDENTIAL,...,CIR,NaN,NaN,NaN,31-Mar-22,Ames City Assessor,2.0,6th Street,42.028348,-93.687718


In [144]:
df_latlong_corrections = pd.read_csv("df_RE_latlongs2019_corrections.csv", index_col = 0)
df_latlong_corrections

,Prop_Addr,reverse_lookup_rd,Lat,Long
11371,5116 SPRINGBROOK CIR,6th Street,42.028568,-93.687674
11373,5122 SPRINGBROOK CIR,6th Street,42.028348,-93.687718


In [150]:
Ames_notebook_state = {}
Ames_notebook_state["df_RE_latlong2019"] = (df_RE_latlongs, "latlong lookup for 2019 using geoapify and reverse lookup using nominatim with corrections done by hand")

In [152]:
pickle.dump(Ames_notebook_state, open("Ames_notebook_state.pkl", "wb"))

In [176]:
#This should be added into the function find_min_dist
#df_RE_latlongs.Lat = df_RE_latlongs.Lat.astype(float)
#df_RE_latlongs.Long = df_RE_latlongs.Long.astype(float)
df_RE_latlongs.index.name = 'SaleID'
df_RE_latlongs.reset_index(inplace = True)

In [177]:
df_min_dist = find_min_dist(df_RE_latlongs, df_EssentialLatLong)
df_min_dist.filter(regex = "_dist$").join(df_min_dist.SalePrice)

,Arts_dist,Elem_School_dist,Golf_dist,Gym_dist,High_School_dist,Historic_dist,ISU_dist,Library_dist,Medical_dist,Movies_dist,Organic_Groceries_dist,Park_dist,Recreation_dist,Religion_dist,Restaurant_dist,Shopping_dist,Spa_dist,Sports_Venue_dist,SalePrice
0,3.621211,1.753512,1.169253,1.899291,2.501688,3.354657,3.284050,3.570212,3.207433,4.464579,1.114132,0.643165,2.238390,1.246691,1.837750,1.839250,1.815357,4.207073,125000.0
1,3.621211,1.753512,1.169253,1.899291,2.501688,3.354657,3.284050,3.570212,3.207433,4.464579,1.114132,0.643165,2.238390,1.246691,1.837750,1.839250,1.815357,4.207073,100000.0
2,3.710401,1.908894,1.462515,2.147550,2.679027,3.445747,3.506823,3.658191,3.276203,4.568679,1.402202,0.465953,2.269335,1.549200,2.085425,1.915365,1.898055,4.393030,399000.0
3,3.710715,1.908000,1.457711,2.145128,2.677882,3.446017,3.504953,3.658521,3.276795,4.568804,1.398923,0.468996,2.270374,1.544397,2.083007,1.915682,1.898271,4.391834,36000.0
4,3.720136,1.904619,1.405491,2.124316,2.671482,3.454992,3.490261,3.668111,3.289235,4.576120,1.368188,0.507822,2.287727,1.492540,2.062262,1.925499,1.907006,4.384658,36000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
896,2.719105,1.797315,1.914441,1.734158,3.894992,2.810964,3.522879,2.772280,3.179553,1.852297,2.821734,2.440837,2.838125,2.543357,1.893270,2.389929,2.663162,2.487271,175000.0
897,2.719870,1.799802,1.916091,1.738524,3.896772,2.812098,3.525571,2.773035,3.180122,1.853269,2.823216,2.444012,2.839262,2.540022,1.897084,2.390592,2.664571,2.490224,137500.0
898,2.728581,1.830410,1.936150,1.793071,3.918086,2.825415,3.558458,2.781614,3.186275,1.864862,2.840913,2.483140,2.852622,2.497452,1.944416,2.398106,2.681381,2.526537,200000.0
899,2.740954,1.871015,1.963501,1.863047,3.946016,2.843593,3.601121,2.793787,3.195152,1.881629,2.864638,2.533937,2.870840,2.443346,2.005533,2.409177,2.704067,2.573810,203000.0


In [178]:
df_min_dist.filter(regex = "_dist$").join(df_min_dist.SalePrice).describe()

,Arts_dist,Elem_School_dist,Golf_dist,Gym_dist,High_School_dist,Historic_dist,ISU_dist,Library_dist,Medical_dist,Movies_dist,Organic_Groceries_dist,Park_dist,Recreation_dist,Religion_dist,Restaurant_dist,Shopping_dist,Spa_dist,Sports_Venue_dist,SalePrice
count,901.000000,901.000000,901.000000,901.000000,901.000000,901.000000,901.000000,901.000000,901.000000,901.000000,901.000000,901.000000,901.000000,901.000000,901.000000,901.000000,901.000000,901.000000,901.000000
mean,1.812260,1.669993,1.521361,0.969215,1.865280,2.194310,1.845845,1.674568,2.363190,2.779018,1.629885,1.785282,1.196070,1.378757,1.392391,1.945837,1.195808,2.230937,226669.041065
std,0.887435,0.950541,0.553034,0.481501,0.918021,1.077520,0.690849,0.837985,1.180465,1.048824,0.999990,0.698322,0.681001,0.649349,0.808266,1.151792,0.633684,0.883734,109469.497483
min,0.118647,0.018421,0.186356,0.016378,0.168045,0.023081,0.601996,0.069744,0.034280,0.345970,0.074869,0.143194,0.080871,0.109660,0.129735,0.131424,0.152513,0.203581,4350.000000
25%,1.083073,0.976900,1.170868,0.645296,1.100829,1.451964,1.330127,0.931484,1.438250,1.823332,0.816905,1.342425,0.605248,0.857544,0.749283,0.958395,0.685229,1.671844,158855.000000
50%,1.874251,1.424420,1.447482,0.934990,1.678778,2.297859,1.823522,1.708547,2.498054,2.869750,1.331813,1.851029,1.138358,1.386585,1.173669,1.835460,1.073824,2.253900,213000.000000
75%,2.554475,2.385538,1.821011,1.322722,2.659455,3.054768,2.302530,2.359493,3.269466,3.634900,2.482519,2.207871,1.777835,1.837417,2.027562,2.854803,1.733856,2.855446,282240.000000
max,3.757390,3.658818,2.891720,2.149028,3.946016,4.019225,3.601121,3.705834,4.325420,4.607510,3.693157,3.352572,2.870840,2.753588,3.139393,4.235085,2.704067,4.393030,771530.000000


## Driving time Lookups Section

In [32]:
df_cross = pd.read_csv("df_cross.csv", index_col = 0)

/var/folders/2n/5k95xl754cb_szsgjkmj33k80000gn/T/ipykernel_80585/2705128043.py:1: DtypeWarning: Columns (9,10,35,36,39,40,84,88,89) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cross = pd.read_csv("df_cross.csv", index_col = 0)


In [33]:
df_mins = df_cross.loc[df_cross.groupby(["Service", "SaleID"]).dist.idxmin()]

In [ ]:
# #Below code is from geoapify website

# import requests

# # Replace YOUR_API_KEY with your actual API key. Sign up and get an API key on https://www.geoapify.com/ 
# API_KEY = "c65731ef1fc94fd3aba9a53df17c2366"

# # Define the address to geocode
# address = "1505 LITTLE BLUESTEM CT UNIT 101, Ames, Iowa"

# def get_geoapify (address):
#     # Build the API URL
#     url = f"https://api.geoapify.com/v1/geocode/search?text={address}&limit=1&apiKey={API_KEY}"

#     # Send the API request and get the response
#     response = requests.get(url)

#     # Check the response status code
#     if response.status_code == 200:
#         # Parse the JSON data from the response
#         data = response.json()

#         # Extract the first result from the data
#         result = data["features"][0]

#         # Extract the latitude and longitude of the result
#         latitude = result["geometry"]["coordinates"][1]
#         longitude = result["geometry"]["coordinates"][0]

#         print(f"Latitude: {latitude}, Longitude: {longitude}")
#     else:
#         print(f"Request failed with status code {response.status_code}")
#     return f"{latitude}, {longitude}"

In [34]:
# to_biz = {}
# from_biz = {}

# for i in OSMR_list:
#     to_biz[i] = df_mins.query(f"Service == '{i}'").apply(lambda x: OSMR(x.Lat_biz, x.Long_biz, x.Lat_house, x.Long_house), axis = 1)
    
# pd.DataFrame(to_biz)

# from_biz['Spa'] = df_mins.query("Service == 'Spa'").apply(lambda x: OSMR(x.Lat_biz, x.Long_biz, x.Lat_house, x.Long_house), axis = 1)
# from_biz

In [35]:
from_biz_final = pickle.load(open("from_biz_final.pkl", 'rb'))
from_biz_final

,Golf,Gym,Arts,Elem_School,Spa,High_School,Historic,ISU,Library,Medical,Movies,Organic_Groceries,Park,Recreation,Religion,Restaurant,Shopping,Sports_Venue
1,NaN,NaN,"[[633.1, 6840.0]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,"[[340.3, 4065.6]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"[[131.2, 1489.0]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,"[[288.2, 3129.1]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,NaN,NaN,NaN,NaN,"[[500.7, 5288.8]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25943,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[[273.6, 4091.9]]",NaN
25945,NaN,NaN,NaN,NaN,"[[363.9, 4665.5]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25947,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[[627.0, 7346.6]]"
25949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[[376.3, 4907.1]]",NaN,NaN


In [36]:
from_biz_stacked = from_biz_final.stack().reset_index().set_axis(["df_cross_index", "Service", "time_dist"], axis = 1)

from_biz_temp = from_biz_stacked.join(from_biz_stacked
                                            .apply(lambda x: f"{x.time_dist[0][0]}, {x.time_dist[0][1]}", axis = 1)
                                            .str.split(", ", expand = True)
                                            .set_axis(["time_secs", "driving_dist_meters"], axis = 1)
                                            .apply(lambda x: x.astype(float))
                                    ).drop("time_dist", axis = 1)

In [37]:
df_driv_final = {}

for k, f in zip(["from_time", "from_driv", "dist"], ["time_secs", "driving_dist_meters", "dist"]):
        df_driv_final[k] = (df_mins
                                .join(from_biz_temp.set_index("df_cross_index"), rsuffix = "_driv")
                                .query("Service == Service_driv")
                                .drop("Service_driv", axis = 1)
                                .pivot(index = ["SaleID", "Lat_house", "Long_house"], columns = "Service", values = f)
                                .reset_index()
                                .set_index("SaleID").join(df_mins[["SaleID", "SalePrice"]].drop_duplicates().set_index("SaleID").SalePrice)
                        )
        
df_driv_final["from_driv"] #.set_index("SaleID").join(df_mins[["SaleID", "SalePrice"]].drop_duplicates().set_index("SaleID").SalePrice)

,Lat_house,Long_house,Arts,Elem_School,Golf,Gym,High_School,Historic,ISU,Library,...,Movies,Organic_Groceries,Park,Recreation,Religion,Restaurant,Shopping,Spa,Sports_Venue,SalePrice
SaleID,,,,,,,,,,,,,,,,,,,,,
2,42.064191,-93.662310,6840.0,4065.6,1489.0,3129.1,5288.8,7728.4,5622.6,6682.4,...,9956.7,3010.9,6083.0,5578.4,1184.7,3112.5,5076.9,4722.2,7714.0,78000.0
3,42.064191,-93.662310,6840.0,4065.6,1489.0,3129.1,5288.8,7728.4,5622.6,6682.4,...,9956.7,3010.9,6083.0,5578.4,1184.7,3112.5,5076.9,4722.2,7714.0,408905.0
18,42.063979,-93.662658,6860.7,4086.2,1509.7,3149.8,5309.5,7749.1,5643.3,6703.0,...,9977.4,3031.6,6103.7,5599.1,1205.3,3133.2,5097.6,4742.8,7734.6,434066.0
19,42.063784,-93.662604,6882.6,4108.2,1531.6,3171.8,5331.5,7771.1,5665.3,6725.0,...,9999.4,3053.5,6125.7,5621.1,1227.3,3155.2,5119.5,4764.8,7756.6,88000.0
20,42.063575,-93.662609,6905.8,4131.3,1554.7,3194.9,5354.6,7794.2,5688.4,6748.1,...,10022.5,3076.7,6148.8,5644.2,1250.4,3178.3,5142.6,4787.9,7779.7,88000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22743,41.986503,-93.606026,4801.4,3859.6,3721.5,5334.0,7854.0,5458.3,8031.8,4886.9,...,3366.1,6029.1,5213.3,5415.7,7948.9,4099.4,4165.9,5221.6,5512.4,220000.0
22749,41.986496,-93.603619,5001.0,4059.2,3921.1,5533.6,8053.6,5657.9,8231.4,5086.5,...,3565.7,6228.7,5412.9,5615.3,8148.5,4299.0,4365.4,5421.2,5712.0,217500.0
22756,41.986499,-93.602049,5131.0,4189.3,4051.1,5663.7,8183.6,5788.0,8361.4,5216.6,...,3695.7,6358.7,5543.0,5745.4,8278.6,4429.1,4495.5,5551.2,5842.0,229000.0


In [ ]:
pickle.dump(df_driv_final, open("df_driv_final.pkl", 'wb'))